# 探索性資料分析 (Exploratory Data Analysis)
## 第三期大腸癌存活預測研究

本筆記本進行資料的初步探索與分析

In [64]:
# 導入必要的套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# 設定路徑
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# 設定繪圖風格
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial']
plt.rcParams['axes.unicode_minus'] = False

print("套件載入完成")

套件載入完成


In [65]:
# 載入資料
data_path = r"/Users/huangshifeng/Desktop/stage_III_colon_surv/Patient_Cohort.xlsx"
df = pd.read_excel(data_path, sheet_name='Patients')

print(f"資料形狀: {df.shape}")
print(f"\n資料欄位:\n{df.columns.tolist()}")
df.head()

資料形狀: (346, 40)

資料欄位:
['Patient_ID', 'Chart_No', 'Dx_Date', 'Dx_Year', 'Age', 'Sex', 'BMI', 'ECOG', 'Tumor_Location', 'Tumor_Location_Group', 'pT_Stage', 'pN_Stage', 'AJCC_Substage', 'LN_Total', 'LN_Positive', 'LNR', 'Histology', 'Differentiation', 'LVI', 'PNI', 'Tumor_Deposits', 'Mucinous_Gt_50', 'Mucinous_Any', 'Signet_Ring', 'MSI_Status', 'Tumor_Size_cm', 'CEA_PreOp', 'Log_CEA_PreOp', 'Radical_Op_Date', 'Op_Procedure', 'PreOp_Albumin', 'Last_FU_Date', 'Recurrence', 'Recurrence_Date', 'Recurrence_Type', 'Death', 'Death_Cause', 'DFS_Months', 'OS_Months', 'Visiting_Staff']


,Patient_ID,Chart_No,Dx_Date,Dx_Year,Age,Sex,BMI,ECOG,Tumor_Location,Tumor_Location_Group,...,PreOp_Albumin,Last_FU_Date,Recurrence,Recurrence_Date,Recurrence_Type,Death,Death_Cause,DFS_Months,OS_Months,Visiting_Staff
0,1,170832,2017/10/16,2017,77,2,24.24,2.0,4,1,...,3.6,2020/03/15,1,2019-04-08,Locoregional,1,3,17.833333,29.57,4
1,2,190783,2017/06/20,2017,82,1,22.23,1.0,7,2,...,NaN,2024/03/20,0,NaT,NaN,0,0,82.530000,82.53,2
2,3,335615,2021/05/15,2021,71,1,30.12,1.0,7,2,...,3.3,2024/05/01,0,NaT,NaN,0,0,36.070000,36.07,1
3,4,458173,2021/11/04,2021,87,2,25.11,1.0,2,1,...,3.8,2024/02/29,0,NaT,NaN,0,0,28.230000,28.23,1
4,5,536710,2020/07/17,2020,90,1,23.39,2.0,7,2,...,3.2,2020/08/19,0,NaT,NaN,1,1,1.170000,1.17,5


In [ ]:
# ====================================
# 資料型態轉換 (Data Type Conversion)
# ====================================

print("原始資料型態:")
print(df.dtypes)
print("\n" + "="*60 + "\n")

# 1. 日期欄位 (Date columns)
# ====================================
print("步驟 1: 轉換日期欄位...")
date_cols = ['Dx_Date', 'Radical_Op_Date', 'Last_FU_Date', 'Recurrence_Date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"  ✓ {col}: 轉換為 datetime64")

print("\n" + "="*60 + "\n")

# 2. 有序類別變數 (Ordered Categorical)
# ====================================
print("步驟 2: 轉換有序類別變數...")

# pT Stage (1-4B)
print(f"\npT_Stage 唯一值: {sorted([str(x) for x in df['pT_Stage'].dropna().unique()])}")
# 注意：實際資料是 '1', '2', '3', '4A', '4B'，不是 'T1', 'T2'
df['pT_Stage'] = df['pT_Stage'].astype(str)  # 確保是字串
T_stage = pd.CategoricalDtype(categories=['1', '2', '3', '4', '4A', '4B'], ordered=True)
df['pT_Stage'] = df['pT_Stage'].replace('nan', pd.NA)  # 處理字串 'nan'
df['pT_Stage'] = df['pT_Stage'].astype(T_stage)
print(f"  ✓ pT_Stage: {df['pT_Stage'].dtype}")
print(f"    非 NaN 數量: {df['pT_Stage'].notna().sum()}")

# pN Stage (1A-2B)
print(f"\npN_Stage 唯一值: {sorted([str(x) for x in df['pN_Stage'].dropna().unique()])}")
# 注意：實際資料是 '1A', '1B', '1C', '2A', '2B'，沒有 'N' 前綴
df['pN_Stage'] = df['pN_Stage'].astype(str)
N_stage = pd.CategoricalDtype(categories=['1A', '1B', '1C', '2A', '2B'], ordered=True)
df['pN_Stage'] = df['pN_Stage'].replace('nan', pd.NA)
df['pN_Stage'] = df['pN_Stage'].astype(N_stage)
print(f"  ✓ pN_Stage: {df['pN_Stage'].dtype}")
print(f"    非 NaN 數量: {df['pN_Stage'].notna().sum()}")

# AJCC Substage (3A, 3B, 3C)
print(f"\nAJCC_Substage 唯一值: {sorted([str(x) for x in df['AJCC_Substage'].dropna().unique()])}")
df['AJCC_Substage'] = df['AJCC_Substage'].astype(str)
AJCC_Stage = pd.CategoricalDtype(categories=['3A', '3B', '3C'], ordered=True)
df['AJCC_Substage'] = df['AJCC_Substage'].replace('nan', pd.NA)
df['AJCC_Substage'] = df['AJCC_Substage'].astype(AJCC_Stage)
print(f"  ✓ AJCC_Substage: {df['AJCC_Substage'].dtype}")
print(f"    非 NaN 數量: {df['AJCC_Substage'].notna().sum()}")

# Differentiation (分化程度)
# 注意：實際資料是數字 1, 2, 3, 4, 9，不是文字
print(f"\nDifferentiation 唯一值: {sorted([x for x in df['Differentiation'].dropna().unique()])}")
# 1=Well, 2=Moderate, 3=Poor, 4=Undifferentiated, 9=Unknown
diff_categories = pd.CategoricalDtype(categories=[1, 2, 3, 4, 9], ordered=True)
df['Differentiation'] = df['Differentiation'].astype(diff_categories)
print(f"  ✓ Differentiation: {df['Differentiation'].dtype}")
print(f"    非 NaN 數量: {df['Differentiation'].notna().sum()}")

# ECOG Performance Status
if 'ECOG' in df.columns:
    print(f"\nECOG 唯一值: {sorted(df['ECOG'].dropna().unique())}")
    ecog_categories = pd.CategoricalDtype(categories=[0, 1, 2, 3, 4], ordered=True)
    df['ECOG'] = df['ECOG'].astype(ecog_categories)
    print(f"  ✓ ECOG: {df['ECOG'].dtype}")

print("\n" + "="*60 + "\n")

# 3. 二元變數 (Binary variables: 0/1)
# ====================================
print("步驟 3: 轉換二元變數...")

# 這些欄位在原始資料中已經是 0/1 格式
binary_cols = {
    'LVI': 'Int64',           # Lymphovascular Invasion (0/1)
    'PNI': 'Int64',           # Perineural Invasion (0/1)
    'Tumor_Deposits': 'Int64', # Tumor Deposits (0/1)
    'Mucinous_Gt_50': 'Int64', # Mucinous >50% (0/1)
    'Mucinous_Any': 'Int64',   # Any Mucinous (0/1)
    'Signet_Ring': 'Int64',    # Signet Ring (0/1, 多為 NaN)
    'Recurrence': 'Int64',     # 復發 (0/1)
    'Death': 'Int64'           # 死亡 (0/1)
}

for col, dtype in binary_cols.items():
    if col in df.columns:
        unique_vals = sorted([str(x) for x in df[col].dropna().unique()])
        print(f"\n{col} 唯一值: {unique_vals}")
        # 轉換為 Int64（保留 NaN）
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(dtype)
        count_0 = (df[col] == 0).sum()
        count_1 = (df[col] == 1).sum()
        count_na = df[col].isna().sum()
        print(f"  ✓ {col}: 0={count_0}, 1={count_1}, NaN={count_na}")

print("\n" + "="*60 + "\n")

# 4. 無序類別變數 (Unordered Categorical)
# ====================================
print("步驟 4: 轉換無序類別變數...")

# 定義無序類別欄位
categorical_cols = {
    'Sex': 'category',                    # 1=男/2=女（需確認）
    'Tumor_Location': 'category',         # 腫瘤位置
    'Tumor_Location_Group': 'category',   # 腫瘤位置分組
    'Histology': 'category',              # 組織型態
    'MSI_Status': 'category',             # MSI-H, MSS
    'Op_Procedure': 'category',           # 手術方式
    'Visiting_Staff': 'category',         # 主治醫師
    'Recurrence_Type': 'category',        # 復發型態
    'Death_Cause': 'category'             # 死因
}

for col, dtype in categorical_cols.items():
    if col in df.columns:
        df[col] = df[col].astype(dtype)
        unique_count = df[col].nunique()
        non_na_count = df[col].notna().sum()
        print(f"  ✓ {col}: {unique_count} 個類別 (非 NaN: {non_na_count})")

print("\n" + "="*60 + "\n")

# 5. 數值變數 (Numeric variables)
# ====================================
print("步驟 5: 確認數值變數...")

numeric_cols = [
    'Age', 'BMI', 'LN_Total', 'LN_Positive', 'LNR', 
    'Tumor_Size_cm', 'CEA_PreOp', 'Log_CEA_PreOp', 
    'PreOp_Albumin', 'DFS_Months', 'OS_Months', 'Dx_Year'
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"  ✓ {col}: {df[col].dtype}")

print("\n" + "="*60 + "\n")

# 6. 識別碼保持為字串 (Identifiers as string)
# ====================================
print("步驟 6: 確認識別碼欄位...")
id_cols = ['Patient_ID', 'Chart_No']
for col in id_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)
        print(f"  ✓ {col}: {df[col].dtype}")

print("\n" + "="*60 + "\n")
print("✅ 資料型態轉換完成！\n")

# 顯示轉換後的資料型態摘要
print("轉換後的資料型態摘要:")
print(df.dtypes)
print(f"\n資料形狀: {df.shape}")



原始資料型態:
Patient_ID                       int64
Chart_No                         int64
Dx_Date                         object
Dx_Year                          int64
Age                              int64
Sex                              int64
BMI                            float64
ECOG                           float64
Tumor_Location                   int64
Tumor_Location_Group             int64
pT_Stage                        object
pN_Stage                        object
AJCC_Substage                   object
LN_Total                         int64
LN_Positive                      int64
LNR                            float64
Histology                        int64
Differentiation                  int64
LVI                            float64
PNI                            float64
Tumor_Deposits                   int64
Mucinous_Gt_50                   int64
Mucinous_Any                     int64
Signet_Ring                    float64
MSI_Status                      object
Tumor_Size_cm    

In [ ]:
# ====================================
# 儲存處理後的資料
# ====================================

# 建立輸出目錄
output_dir = project_root / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

# 儲存為 CSV
output_csv = output_dir / 'df_cleaned.csv'
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"✅ 已儲存處理後資料至: {output_csv}")

# 儲存為 pickle（完整保留資料型態）
output_pkl = output_dir / 'df_cleaned.pkl'
df.to_pickle(output_pkl)
print(f"✅ 已儲存 pickle 檔案至: {output_pkl}")

# 顯示摘要資訊
print("\n" + "=" * 60)
print("📊 資料儲存完成摘要:")
print("=" * 60)
print(f"總樣本數: {len(df)}")
print(f"總欄位數: {len(df.columns)}")
print(f"檔案大小: {output_csv.stat().st_size / 1024:.2f} KB (CSV)")
print(f"檔案大小: {output_pkl.stat().st_size / 1024:.2f} KB (pickle)")
print("\n💡 提示:")
print("  - CSV: 可用 Excel 開啟，但會失去資料型態資訊")
print("  - pickle: 完整保留所有資料型態，建議用於後續分析")


In [67]:
# ====================================
# 資料型態驗證與檢查
# ====================================

print("資料型態驗證結果:")
print("=" * 60)

# 檢查日期欄位
print("\n【日期欄位】")
date_cols = ['Dx_Date', 'Radical_Op_Date', 'Last_FU_Date', 'Recurrence_Date']
for col in date_cols:
    if col in df.columns:
        print(f"  {col}: {df[col].dtype}")
        print(f"    - 範圍: {df[col].min()} 至 {df[col].max()}")
        print(f"    - 缺失: {df[col].isna().sum()} ({df[col].isna().sum()/len(df)*100:.1f}%)")

# 檢查有序類別變數
print("\n【有序類別變數】")
ordered_categorical = ['pT_Stage', 'pN_Stage', 'AJCC_Substage', 'Differentiation', 'ECOG']
for col in ordered_categorical:
    if col in df.columns:
        print(f"\n  {col}:")
        print(f"    類別: {df[col].cat.categories.tolist()}")
        print(f"    分佈: \n{df[col].value_counts().sort_index()}")

# 檢查二元變數
print("\n【二元變數】")
binary_cols = ['Recurrence', 'Death']
for col in binary_cols:
    if col in df.columns:
        print(f"\n  {col}:")
        print(f"    分佈: {df[col].value_counts()}")
        print(f"    缺失: {df[col].isna().sum()}")

# 檢查關鍵數值變數的範圍
print("\n【關鍵數值變數】")
key_numeric = ['Age', 'BMI', 'LNR', 'CEA_PreOp', 'PreOp_Albumin']
for col in key_numeric:
    if col in df.columns:
        print(f"\n  {col}:")
        print(f"    Min: {df[col].min():.2f}, Max: {df[col].max():.2f}")
        print(f"    Mean: {df[col].mean():.2f}, Median: {df[col].median():.2f}")
        print(f"    缺失: {df[col].isna().sum()} ({df[col].isna().sum()/len(df)*100:.1f}%)")

print("\n" + "=" * 60)
print("✅ 資料型態驗證完成！")


資料型態驗證結果:

【日期欄位】
  Dx_Date: datetime64[ns]
    - 範圍: 2017-01-14 00:00:00 至 2022-01-07 00:00:00
    - 缺失: 0 (0.0%)
  Radical_Op_Date: datetime64[ns]
    - 範圍: 2017-01-23 00:00:00 至 2022-02-04 00:00:00
    - 缺失: 0 (0.0%)
  Last_FU_Date: datetime64[ns]
    - 範圍: 2017-10-21 00:00:00 至 2024-05-04 00:00:00
    - 缺失: 0 (0.0%)
  Recurrence_Date: datetime64[ns]
    - 範圍: 2017-10-17 00:00:00 至 2023-06-06 00:00:00
    - 缺失: 258 (74.6%)

【有序類別變數】

  pT_Stage:
    類別: ['1', '2', '3', '4', '4A', '4B']
    分佈: 
pT_Stage
1      15
2      30
3     243
4       0
4A     44
4B     14
Name: count, dtype: int64

  pN_Stage:
    類別: ['1A', '1B', '1C', '2A', '2B']
    分佈: 
pN_Stage
1A     97
1B    109
1C     19
2A     64
2B     57
Name: count, dtype: int64

  AJCC_Substage:
    類別: ['3A', '3B', '3C']
    分佈: 
AJCC_Substage
3A     40
3B    237
3C     69
Name: count, dtype: int64

  Differentiation:
    類別: [1, 2, 3, 4, 9]
    分佈: 
Differentiation
1      4
2    312
3     24
4      5
9      1
Name: count, dtype

In [51]:
# 使用 ydata-profiling 進行自動化 EDA
try:
    from ydata_profiling import ProfileReport
    
    # 生成完整的資料分析報告
    print("正在生成 ydata-profiling 報告...")
    profile = ProfileReport(
        df, 
        title="第三期大腸癌存活預測 - 資料分析報告",
        explorative=True,
        minimal=False
    )
    
    # 儲存 HTML 報告
    report_path = project_root / 'data_profile_report.html'
    profile.to_file(report_path)
    print(f"報告已儲存至: {report_path}")
    
    # 在 Jupyter 中顯示報告
    # profile.to_notebook_iframe()
    
except ImportError:
    print("未安裝 ydata-profiling，請執行: pip install ydata-profiling")
    print("繼續使用基本的 EDA 方法...")

正在生成 ydata-profiling 報告...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:00<00:00, 701975.56it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

報告已儲存至: /Users/huangshifeng/Desktop/stage_III_colon_surv/data_profile_report.html


In [68]:
# ====================================
# 基本統計資訊
# ====================================

print("=" * 60)
print("資料基本資訊:")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("描述性統計 (數值變數):")
print("=" * 60)
print(df.describe())

print("\n" + "=" * 60)
print("類別變數摘要:")
print("=" * 60)
categorical_summary = df.select_dtypes(include=['category', 'object']).describe()
print(categorical_summary)

資料基本資訊:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Patient_ID            346 non-null    object        
 1   Chart_No              346 non-null    object        
 2   Dx_Date               346 non-null    datetime64[ns]
 3   Dx_Year               346 non-null    int64         
 4   Age                   346 non-null    int64         
 5   Sex                   346 non-null    category      
 6   BMI                   342 non-null    float64       
 7   ECOG                  328 non-null    category      
 8   Tumor_Location        346 non-null    category      
 9   Tumor_Location_Group  346 non-null    category      
 10  pT_Stage              346 non-null    category      
 11  pN_Stage              346 non-null    category      
 12  AJCC_Substage         346 non-null    category      
 13  LN_Total    

In [69]:
# 檢查缺失值
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    '缺失數量': missing_data,
    '缺失比例(%)': missing_percent
})

missing_df = missing_df[missing_df['缺失數量'] > 0].sort_values('缺失數量', ascending=False)

if len(missing_df) > 0:
    print("缺失值統計:")
    print(missing_df)
else:
    print("沒有缺失值")

缺失值統計:
                 缺失數量    缺失比例(%)
Signet_Ring       344  99.421965
Recurrence_Date   258  74.566474
Recurrence_Type   258  74.566474
PreOp_Albumin      57  16.473988
ECOG               18   5.202312
CEA_PreOp           6   1.734104
Log_CEA_PreOp       6   1.734104
BMI                 4   1.156069
LVI                 4   1.156069
PNI                 3   0.867052
MSI_Status          2   0.578035
Tumor_Size_cm       2   0.578035


In [ ]:
# 視覺化缺失值
if len(missing_df) > 0:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_df['缺失比例(%)'], y=missing_df.index)
    plt.xlabel('缺失比例 (%)')
    plt.ylabel('變數')
    plt.title('各變數缺失值比例')
    plt.tight_layout()
    plt.show()

In [70]:
# 數值變數分佈視覺化
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 0:
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows*4))
    axes = axes.flatten() if n_rows > 1 else [axes]
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            df[col].hist(bins=30, ax=axes[idx], edgecolor='black')
            axes[idx].set_title(f'{col} 分佈')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('頻率')
    
    # 隱藏多餘的子圖
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/828290396.py:22: UserWarning: Glyph 38971 (\N{CJK UNIFIED IDEOGRAPH-983B}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/828290396.py:22: UserWarning: Glyph 29575 (\N{CJK UNIFIED IDEOGRAPH-7387}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/828290396.py:22: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/828290396.py:22: UserWarning: Glyph 20296 (\N{CJK UNIFIED IDEOGRAPH-4F48}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/828290396.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [71]:
# 類別變數分佈
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols) > 0:
    for col in categorical_cols:
        plt.figure(figsize=(10, 5))
        df[col].value_counts().plot(kind='bar')
        plt.title(f'{col} 分佈')
        plt.xlabel(col)
        plt.ylabel('數量')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:12: UserWarning: Glyph 25976 (\N{CJK UNIFIED IDEOGRAPH-6578}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:12: UserWarning: Glyph 37327 (\N{CJK UNIFIED IDEOGRAPH-91CF}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:12: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:12: UserWarning: Glyph 20296 (\N{CJK UNIFIED IDEOGRAPH-4F48}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/4y/l6fx63p966z4_0myl5b68p8h0000gn/T/ipykernel_9743/455771210.py:12:

In [ ]:
# 相關性矩陣 (僅數值變數)
if len(numeric_cols) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_cols].corr()
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('變數相關性矩陣')
    plt.tight_layout()
    plt.show()

In [ ]:
# ====================================
# 儲存處理後的資料
# ====================================

# 建立 data 目錄（如果不存在）
data_dir = project_root / 'data'
raw_dir = data_dir / 'raw'
processed_dir = data_dir / 'processed'

processed_dir.mkdir(parents=True, exist_ok=True)

# 儲存為 CSV（保留資料型態資訊）
output_path = processed_dir / 'cleaned_data.csv'
df.to_csv(output_path, index=False)
print(f"✅ 已儲存處理後資料至: {output_path}")

# 儲存為 pickle（完整保留資料型態，包含 category）
pickle_path = processed_dir / 'cleaned_data.pkl'
df.to_pickle(pickle_path)
print(f"✅ 已儲存 pickle 檔案至: {pickle_path}")

# 儲存資料型態資訊
dtype_info = pd.DataFrame({
    'Column': df.columns,
    'Dtype': df.dtypes.astype(str),
    'Non_Null_Count': df.notna().sum(),
    'Null_Count': df.isna().sum(),
    'Null_Percentage': (df.isna().sum() / len(df) * 100).round(2)
})

dtype_path = processed_dir / 'data_types_info.csv'
dtype_info.to_csv(dtype_path, index=False)
print(f"✅ 已儲存資料型態資訊至: {dtype_path}")

print("\n" + "=" * 60)
print("📊 資料處理完成摘要:")
print("=" * 60)
print(f"總樣本數: {len(df)}")
print(f"總欄位數: {len(df.columns)}")
print(f"\n資料型態分布:")
print(df.dtypes.value_counts())
print("\n探索性資料分析完成！請繼續進行資料預處理步驟。")


In [ ]:
# 存活分析相關視覺化 (如果有相關欄位)
# 這部分需要根據實際資料欄位調整
print("\n探索性資料分析完成！")
print("請根據資料特性調整後續分析步驟")

In [55]:
df.dtypes

Patient_ID                       int64
Chart_No                         int64
Dx_Date                 datetime64[ns]
Dx_Year                          int64
Age                              int64
Sex                              int64
BMI                            float64
ECOG                           float64
Tumor_Location                   int64
Tumor_Location_Group             int64
pT_Stage                      category
pN_Stage                      category
AJCC_Substage                 category
LN_Total                         int64
LN_Positive                      int64
LNR                            float64
Histology                        int64
Differentiation                  int64
LVI                            float64
PNI                            float64
Tumor_Deposits                   int64
Mucinous_Gt_50                   int64
Mucinous_Any                     int64
Signet_Ring                    float64
MSI_Status                      object
Tumor_Size_cm            